# Génération de clé privée et publique

In [4805]:
import sys
sys.version

'3.9.15 (main, Nov  4 2022, 11:11:31) \n[Clang 12.0.0 ]'

In [4806]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4807]:
pip install falcon

Note: you may need to restart the kernel to use updated packages.


On génère la clé privé en sélectionnant de façon pseudo-aléatoire un nombre entre 1 et 2^256 (environ 1,175*10^77), notre clé privé est ce nombre aléatoire. 

La sécurité provient du fait qu'il est quasiment impossible de sélectionner le même chiffre par hasard.

In [4808]:
from genKeys import privateKey_gen
import math
privateKey = privateKey_gen()
print('La clé privée en valeur décimale est',privateKey)
taille_random = int(math.log10(privateKey)) + 1
print("Taille de la clé privée en valeur décimale :", taille_random)

La clé privée en valeur décimale est 108008149035590916480043517742926194702737936367129282726684337901072797887088
Taille de la clé privée en valeur décimale : 78


On va maintenant calculer la taille de la clé en octet car cela sera nécessaire pour le mettre en argument de la courbe elliptique.

In [4809]:
from genKeys import sizeof
sizeof(privateKey)

32

La fonction compress permet de compresser la clé pour économiser de l'espace de stockage, transmettre plus rapidement, optimiser ou réduire les coûts.

In [4810]:
from genKeys import compress
privateKeyCompressed = compress(privateKey)
privateKeyCompressed


27650086153111274618891140542189105843900911709985096378031190502674636259094529

In [4811]:
sizeof(privateKeyCompressed)

33

La fonction get_publicKey_points permet d'obtenir les points x et y obtenu par le passage de la clé privé dans la courbe elliptique SECP256k1.

Ces deux valeurs nous permettrons de définir la clé publique

In [4812]:
from genKeys import get_publicKey_points
(x,y) = get_publicKey_points(privateKeyCompressed)
(x,y)

(mpz(14446837025792443932741176461993223577041741941234793559591269711067511692433),
 mpz(58010717092803112901553175593431859198588074653439036342698364984128300193129))

In [4813]:
from genKeys import mpz_to_bytes
print(mpz_to_bytes(x))
print(mpz_to_bytes(y))

1ff09eb78a5cf78d69051e7a9a1d200bf4a082958a80446645fd2a6d0b429c91
8040e6fe76d160a102aa0618b28938cbbfc83f6a334fff8a91df905510651569


In [4814]:
from genKeys import get_publicKey
publicKeyBytes = get_publicKey(x,y)
publicKeyBytes
print("Taille de la clé publique en valeur héxadécimale:", publicKeyBytes)

Taille de la clé publique en valeur héxadécimale: 031ff09eb78a5cf78d69051e7a9a1d200bf4a082958a80446645fd2a6d0b429c91


In [4815]:
from genKeys import sizeof
sizeof(int(publicKeyBytes, 16))

33

Si on devait avoir accès à notre clé privé elle serait conservé en base 58 (enlève les O,0,l,I pour éviter les erreurs de transcriptions)(commence par un 5,K ou L). 

Donc principalement utilisé pour la clé privé et l'adresse.

La compression est plus utilisé pour la clé publique afin de réduire la taille de la clé lors des transactions.

In [4816]:
from genKeys import base58_from_int, int_from_base58
privateKeyBase58 = base58_from_int(privateKey)
print(privateKeyBase58)
print("Taille de la clé privée en base 58 :",len(privateKeyBase58))
intKey = int_from_base58(privateKeyBase58)
print(intKey)


H595RiLtpj7iNgh436B1YXvifayvyQwMd7uoYF3f5aj9
Taille de la clé privée en base 58 : 44
108008149035590916480043517742926194702737936367129282726684337901072797887088


In [4817]:
from genKeys import pk_to_address
address = pk_to_address(publicKeyBytes)
print("L'adresse bitcoin pour envoyer et recevoir est ", address)

L'adresse bitcoin pour envoyer et recevoir est  7hf4tLqzCeuVwy9KTUdKNbFXaWRoS2Yce


In [4818]:
from genKeys import generate_keys_address
generate_keys_address()

['pwpghWdnnzivNyLgfEqHCyaTQTcmgWb4h65bdH3XFbn',
 '03553f788710158444a5be95f31abea3f389ca80dd3325a5dd4872e562e2722f92',
 '5qXqP3dqdjknhedXFBAdkMwXwiAuVvaYw']

In [4819]:
from genKeys import list_address
list = list_address()
list

(['CsvJNKRVdaHi5ScnyLvNybiLaN8q2EMnShHspRyT8Ys5',
  '5tdAUbM2PWxtVcgwLs8Yj2Ln6CZfYLwnKN1dSxpDcH5f',
  '7mTYgXtSrQr736mGYN1vYpfHoZcXoKsGKjZtUevXGRwX'],
 ['0262455d5adc0b38f696028c32094547b4a6fee4cbe6da16f6a12dbcd46b6fc166',
  '02e63cc092c5353ceaffb92ace71ba64d35b90f656adc1fad5ade61214f42f789f',
  '03651815d703f5e351dc6570296d252dbb00b225716628cd10c6b046e2c8e01c53'],
 ['LUz6VzMb87GMZmFdJcSJZbKwwhe5kGj79',
  '5z1pXfdjaykjSGrxDL9dq7nkMh1LBjnDG',
  'FumKbkKNo7MiixTzNA96eY7xrEeuEbPEk'])

In [4820]:
from genKeys import get_address
address_test = get_address(list)
address_test

'CsvJNKRVdaHi5ScnyLvNybiLaN8q2EMnShHspRyT8Ys5'

In [4821]:
from genKeys import generate_signature, pkscript_hash
message_hash = pkscript_hash()
signature = generate_signature(privateKey, message_hash)
print(signature)

9dbf02fc44c83a26ce18765aa3380bd9a836f0decbeb20231c93ddbd7c8677e60dd7a8568af3a781ce63fdb71932af38052ebe79cddeb252acacdfcf7bb20745


In [4822]:
print(f"Longueur de la clé publique : {len(publicKeyBytes)}")
print(signature)
print(message_hash)
print(publicKeyBytes)

Longueur de la clé publique : 66
9dbf02fc44c83a26ce18765aa3380bd9a836f0decbeb20231c93ddbd7c8677e60dd7a8568af3a781ce63fdb71932af38052ebe79cddeb252acacdfcf7bb20745
b'+\xcd0=\xa0!X\xed\xbe\t\xe4X\x03vwb\xc2G\x0b\xd7\xbdl\x07\x87\x077\xb4y]\x10Lw'
031ff09eb78a5cf78d69051e7a9a1d200bf4a082958a80446645fd2a6d0b429c91


In [4823]:
from genKeys import verify_signature
verify_signature(publicKeyBytes, signature, message_hash)

True

In [4824]:
from genKeys import inputs, outputs
input = inputs(3, 3)
output = outputs(3,3)
print(input)
print(output)


['{\n  "pkscript": "001441b089e98dc4c29ac3c1950acfe7ba3a06b4cf01",\n  "value": 3,\n  "address": "CKSa3RpKRi3meckvjydLqkt1iu1C47g9x",\n  "signature": "0f6c617286376bd044b8474a53b88ba6ae0dedb1395b2c80357dcf8f2026a445c6189a11c87a62b993a1f8f80b8d668c0b403eb54675963606f6e9c584d7e37d",\n  "publicK": "039a197a7578f67c7a2d304a32a4acf418b46d235046fdb73176d0124e0c64d96e"\n}', '{\n  "pkscript": "001441b089e98dc4c29ac3c1950acfe7ba3a06b4cf01",\n  "value": 3,\n  "address": "8MtQdfPJgpR43mAbe19UJ8YAize3ZHkmG",\n  "signature": "e82e058d960ed4fd4f6c3f02c95f5b1de2243d3e05edf6e1ae89dadfcf2c8a2a24bc7e131cb76394a2aa6bb9f969f435dfd3d3855f121fcbcdcc208004769564",\n  "publicK": "03d86e2e2585c810e29db711810f1005f001a7d78b2bb2c629526bd828452ca51c"\n}', '{\n  "pkscript": "001441b089e98dc4c29ac3c1950acfe7ba3a06b4cf01",\n  "value": 3,\n  "address": "6g6UfCBFvNoNGzFZ3Pg7Rmks6AuPxu7Gs",\n  "signature": "f23d7a6af0db56838a3cb4b228a4d2b37bf2fc380ba315d555ef96fac3a2e6f88b7c400dfba3be2d43227df59d296d3391cf6ccc74a3a3f7ce

In [4825]:
from genKeys import create_json_transaction
json = create_json_transaction()
json

'{\n  "  INPUTS  ": [\n    "{\\n  \\"pkscript\\": \\"001441b089e98dc4c29ac3c1950acfe7ba3a06b4cf01\\",\\n  \\"value\\": 3,\\n  \\"address\\": \\"KwzBdY8ZeBdo6M31aCxfvwPdwqjsVPb4i\\",\\n  \\"signature\\": \\"b2a2b370756b9edb0f3c3366100ef8052b5f0dfb3ed319b1554afdfd8abafa3a22a9505e96574f0feaabd853f1648fb8cdeff4995c36e5a0150d865f02f4269e\\",\\n  \\"publicK\\": \\"02195fdaac9ce3f0825097696e82e5c6fdc60ec93e6b15158960177aedc91afa53\\"\\n}"\n  ],\n  "  OUTPUTS  ": [\n    "{\\n  \\"pkscript\\": \\"001441b089e98dc4c29ac3c1950acfe7ba3a06b4cf01\\",\\n  \\"value\\": 2.99748,\\n  \\"address\\": \\"NqBobnvMvjFuNJMVRAQdEMPGsQH6NLEtG\\"\\n}"\n  ],\n  "fee": 0.00252\n}'

In [4826]:
from genKeys import measure_json_file, json_to_binary
measure_json_file(json)

611

In [4827]:
from genKeys import measure_size
measure_size()

611
624
625
611
611
611
611
611
626
611


615.2